In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import os
import warnings
import time
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from torchinfo import summary
import random

warnings.filterwarnings('ignore')

# =============================================================================
# 0. 设置随机数种子
# =============================================================================
def set_seed(seed):
    """设置随机种子以保证结果可复现"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    print(f"随机数种子已设置为: {seed}")

GLOBAL_SEED = 42
set_seed(GLOBAL_SEED)

# =============================================================================
# 1. PSO 算法实现
# =============================================================================
class Particle:
    """
    代表粒子群中的单个粒子。
    """
    def __init__(self, param_space):
        self.param_space = param_space
        self.dims = len(param_space)
        
        # 初始化位置和速度
        self.position = np.array([random.uniform(low, high) for low, high, _ in param_space.values()])
        self.velocity = np.array([random.uniform(-(high-low)/2, (high-low)/2) for low, high, _ in param_space.values()])
        
        # 初始化个体最优
        self.pbest_position = self.position.copy()
        self.pbest_score = float('inf')

    def update_velocity(self, gbest_position, w=0.5, c1=1.5, c2=1.5):
        """更新粒子速度"""
        r1 = np.random.rand(self.dims)
        r2 = np.random.rand(self.dims)
        
        cognitive_velocity = c1 * r1 * (self.pbest_position - self.position)
        social_velocity = c2 * r2 * (gbest_position - self.position)
        
        self.velocity = w * self.velocity + cognitive_velocity + social_velocity

    def update_position(self):
        """更新粒子位置并确保其在边界内"""
        self.position = self.position + self.velocity
        
        # 边界检查
        for i, key in enumerate(self.param_space):
            low, high, _ = self.param_space[key]
            self.position[i] = np.clip(self.position[i], low, high)

    def get_params_dict(self):
        """将粒子的位置向量转换为适用于模型的超参数字典"""
        params = {}
        for i, key in enumerate(self.param_space):
            _, _, param_type = self.param_space[key]
            # 根据参数类型（如整数）进行转换
            if param_type == 'int':
                value = int(round(self.position[i]))
                # 确保kernel_size至少为1
                if key == 'cnn_kernel_size' and value < 1:
                    value = 1
                params[key] = value
            else: # float
                params[key] = self.position[i]
        return params

class PSOOptimizer:
    """
    粒子群优化器主类。
    """
    def __init__(self, objective_function, param_space, n_particles, max_iter, **kwargs):
        self.objective_function = objective_function
        self.param_space = param_space
        self.n_particles = n_particles
        self.max_iter = max_iter
        self.kwargs = kwargs # 用于传递额外参数给目标函数，如data_loaders
        
        self.gbest_position = None
        self.gbest_score = float('inf')
        
        # 初始化粒子群
        self.swarm = [Particle(param_space) for _ in range(n_particles)]

    def optimize(self):
        print("--- 开始PSO优化 ---")
        start_time = time.time()
        
        for i in range(self.max_iter):
            print(f"\n{'='*20} PSO迭代: {i+1}/{self.max_iter} {'='*20}")
            
            for j, particle in enumerate(self.swarm):
                # 1. 获取当前粒子的超参数
                current_params = particle.get_params_dict()
                print(f"\n--- [迭代 {i+1}/{self.max_iter} | 粒子 {j+1}/{self.n_particles}] 正在评估参数: {current_params} ---")
                
                # 2. 运行目标函数（模型训练与评估），获取适应度分数
                score = self.objective_function(current_params, **self.kwargs)
                print(f"--- [迭代 {i+1}/{self.max_iter} | 粒子 {j+1}/{self.n_particles}] 评估得分 (验证损失): {score:.6f} ---")
                
                # 3. 更新个体最优
                if score < particle.pbest_score:
                    particle.pbest_score = score
                    particle.pbest_position = particle.position.copy()
                    print(f"新个体最优! 粒子 {j+1} 更新得分: {score:.6f}")
                
                # 4. 更新全局最优
                if score < self.gbest_score:
                    self.gbest_score = score
                    self.gbest_position = particle.position.copy()
                    print(f"\n{'*'*20} 新全局最优! {'*'*20}")
                    print(f"全局最优得分: {self.gbest_score:.6f}")
                    print(f"全局最优参数: {self.get_best_params_dict()}")
                    print(f"{'*'*50}\n")

            # 5. 更新所有粒子的速度和位置
            if self.gbest_position is not None:
                for particle in self.swarm:
                    particle.update_velocity(self.gbest_position)
                    particle.update_position()
        
        end_time = time.time()
        print("\n--- PSO优化完成 ---")
        print(f"总耗时: {(end_time - start_time)/3600:.2f} 小时")
        print(f"最佳验证损失: {self.gbest_score:.6f}")
        print(f"最佳超参数: {self.get_best_params_dict()}")
        
        return self.get_best_params_dict(), self.gbest_score

    def get_best_params_dict(self):
        """将最佳位置向量转换为超参数字典"""
        params = {}
        for i, key in enumerate(self.param_space):
            _, _, param_type = self.param_space[key]
            if param_type == 'int':
                value = int(round(self.gbest_position[i]))
                if key == 'cnn_kernel_size' and value < 1:
                    value = 1
                params[key] = value
            else:
                params[key] = self.gbest_position[i]
        return params

# =============================================================================
# 2. 文件路径
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"当前设备: {device}")


HIGH_FREQ_DATA_PATH = r''
LOW_FREQ_DATA_PATH = r''
ORIGINAL_DATA_PATH = r''
MODEL_FILE_PATH = 'pso_best_model.pth' # PSO找到的最佳模型保存路径

# --- 固定超参数 ---
N_STEPS_IN, N_STEPS_OUT = 96, 24
TARGET_STEPS = [3, 6, 12, 24]
HIGH_FREQ_FEATURES = 1
WEATHER_FEATURES = 5
LOW_FREQ_FEATURES = 1
BATCH_SIZE = 64
LEARNING_RATE = 0.0001
NUM_HEADS = 8
DROPOUT = 0.2
TRANSFORMER_LAYERS = 2
RELATIVE_POSITION_BUCKETS = 32

# =============================================================================
# 3. 数据预处理函数
# =============================================================================
def time_series_to_supervised_mimo(data, n_in=96, n_out=1, dropnan=True):
    n_vars = 1 if isinstance(data, list) else data.shape[1]
    df = pd.DataFrame(data)
    orig_names = df.columns
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (orig_names[j], i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        names += [('%s(t%s%d)' % (orig_names[j], '' if i==0 else '+', i)) for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# =============================================================================
# 4. 模型架构 
# =============================================================================
# --- 模块 1: 相对位置偏置生成器---
class RelativePositionBias(nn.Module):
    def __init__(self, num_buckets, max_distance, num_heads):
        super().__init__()
        self.num_buckets = num_buckets
        self.max_distance = max_distance
        self.num_heads = num_heads
        self.relative_attention_bias = nn.Embedding(self.num_buckets, self.num_heads)

    @staticmethod
    def _relative_position_bucket(relative_position, bidirectional=True, num_buckets=32, max_distance=128):
        ret = 0
        n = -relative_position
        if bidirectional:
            num_buckets //= 2
            ret += (n < 0).to(torch.long) * num_buckets
            n = torch.abs(n)
        else:
            n = torch.max(n, torch.zeros_like(n))
        max_exact = num_buckets // 2
        is_small = n < max_exact
        val_if_large = max_exact + (torch.log(n.float() / max_exact) / np.log(max_distance / max_exact) * (num_buckets - max_exact)).to(torch.long)
        val_if_large = torch.min(val_if_large, torch.full_like(val_if_large, num_buckets - 1))
        ret += torch.where(is_small, n, val_if_large)
        return ret

    def forward(self, seq_len, device):
        q_pos = torch.arange(seq_len, dtype=torch.long, device=device)
        k_pos = torch.arange(seq_len, dtype=torch.long, device=device)
        rel_pos = k_pos[None, :] - q_pos[:, None]
        rp_bucket = self._relative_position_bucket(rel_pos, bidirectional=True, num_buckets=self.num_buckets, max_distance=self.max_distance)
        bias = self.relative_attention_bias(rp_bucket)
        return bias.permute(2, 0, 1).unsqueeze(0)

# --- 模块 2: UV RPE Transformer块---
class UV_ScaledRPE_Block(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        assert self.head_dim * self.num_heads == self.d_model, "embed_dim must be divisible by num_heads"
        self.qkv_proj = nn.Linear(d_model, d_model * 3)
        self.out_proj = nn.Linear(d_model, d_model)
        self.attn_dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(nn.Linear(d_model, d_ff), nn.GELU(), nn.Dropout(dropout), nn.Linear(d_ff, d_model))
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.rpe_scale_u = nn.Parameter(torch.ones(1, num_heads, 1, 1))
        self.rpe_scale_v = nn.Parameter(torch.ones(1, num_heads, 1, 1))
    def forward(self, x, bias):
        residual = x
        x = self.norm1(x)
        B, L, D = x.shape
        q, k, v = self.qkv_proj(x).chunk(3, dim=-1)
        q = q.view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        final_bias = (bias * self.rpe_scale_u) + (bias * self.rpe_scale_v)
        attn_scores = attn_scores + final_bias
        attn_probs = torch.softmax(attn_scores, dim=-1)
        attn_probs = self.attn_dropout(attn_probs)
        attn_output = torch.matmul(attn_probs, v).transpose(1, 2).contiguous().view(B, L, D)
        attn_output = self.out_proj(attn_output)
        x = residual + self.dropout1(attn_output)
        residual = x
        x = self.norm2(x)
        x = self.ffn(x)
        x = residual + self.dropout2(x)
        return x

# --- 模块 3: 高频分支---
class EnhancedHighFrequencyModel(nn.Module):
    def __init__(self, power_dim, weather_dim, embed_dim, cnn_filters, cnn_kernel_size, n_gru_layers, num_heads, dropout, n_transformer_blocks, num_pos_buckets):
        super().__init__()
        # 确保kernel_size为奇数，以方便对称填充
        if cnn_kernel_size % 2 == 0:
            cnn_kernel_size +=1
        padding = (cnn_kernel_size - 1) // 2

        self.power_conv = nn.Conv1d(in_channels=power_dim, out_channels=cnn_filters, kernel_size=cnn_kernel_size, padding=padding)
        self.power_bigru = nn.GRU(cnn_filters, embed_dim, n_gru_layers, batch_first=True, bidirectional=True, dropout=dropout)
        self.power_fc = nn.Linear(embed_dim * 2, embed_dim)

        self.weather_bigru = nn.GRU(weather_dim, embed_dim, 1, batch_first=True, bidirectional=True)
        self.weather_fc = nn.Linear(embed_dim * 2, embed_dim)

        self.relative_pos_bias_generator = RelativePositionBias(num_buckets=num_pos_buckets, max_distance=N_STEPS_IN, num_heads=num_heads)
        
        self.transformer_blocks = nn.ModuleList()
        for _ in range(n_transformer_blocks):
            block = nn.ModuleDict({
                'uv_scaled_rpe_self_attn': UV_ScaledRPE_Block(d_model=embed_dim, num_heads=num_heads, d_ff=embed_dim*4, dropout=dropout),
                'norm_cross_attn': nn.LayerNorm(embed_dim),
                'cross_attn': nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout, batch_first=True),
                'dropout_cross_attn': nn.Dropout(dropout),
                'norm_ffn_cross': nn.LayerNorm(embed_dim),
                'ffn_cross': nn.Sequential(nn.Linear(embed_dim, embed_dim * 4), nn.GELU(), nn.Dropout(dropout), nn.Linear(embed_dim * 4, embed_dim)),
                'dropout_ffn_cross': nn.Dropout(dropout)
            })
            self.transformer_blocks.append(block)

    def forward(self, x_high_freq, x_weather):
        h_power_conv = self.power_conv(x_high_freq.permute(0, 2, 1)).permute(0, 2, 1)
        h_power_gru = self.power_bigru(h_power_conv)[0]
        h_power = self.power_fc(h_power_gru)
        h_weather = self.weather_fc(self.weather_bigru(x_weather)[0])
        
        relative_bias = self.relative_pos_bias_generator(h_power.size(1), device=h_power.device)

        processed_power = h_power
        for block in self.transformer_blocks:
            processed_power = block['uv_scaled_rpe_self_attn'](processed_power, relative_bias)
            residual = processed_power
            norm_power_for_cross = block['norm_cross_attn'](processed_power)
            cross_attn_output, _ = block['cross_attn'](query=norm_power_for_cross, key=h_weather, value=h_weather)
            processed_power = residual + block['dropout_cross_attn'](cross_attn_output)
            residual = processed_power
            norm_power_for_ffn = block['norm_ffn_cross'](processed_power)
            ffn_output = block['ffn_cross'](norm_power_for_ffn)
            processed_power = residual + block['dropout_ffn_cross'](ffn_output)
            
        return processed_power[:, -1, :]

# --- 低频分支 ---
class LowFrequencyLSTM(nn.Module):
    def __init__(self, input_dim, embed_dim, n_lstm_layers, dropout):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, embed_dim, n_lstm_layers, batch_first=True, dropout=dropout)

    def forward(self, x_low_freq):
        _, (h_n, _) = self.lstm(x_low_freq)
        return h_n[-1]

# --- 顶层模型---
class ICIAL_UV_RPE_Model(nn.Module):
    def __init__(self, target_steps_names, power_dim, weather_dim, low_freq_dim, 
                 embed_dim, n_gru_layers, n_lstm_layers, cnn_filters, cnn_kernel_size, # 可调参数
                 num_heads, dropout, transformer_layers, num_pos_buckets): # 固定参数
        super().__init__()
        # 验证 embed_dim 是否能被 num_heads 整除
        if embed_dim % num_heads != 0:
            # 调整 embed_dim 为最接近的、能被 num_heads 整除的数
            embed_dim = round(embed_dim / num_heads) * num_heads
            print(f"警告: embed_dim ({params['units']}) 不能被 num_heads ({num_heads}) 整除。已自动调整为 {embed_dim}。")
        
        self.target_steps_names = target_steps_names
        self.quantiles_to_predict = sorted(list(set([0.5, 0.025, 0.975, 0.05, 0.95, 0.1, 0.9, 0.15, 0.85, 0.2, 0.8, 0.075, 0.925])))
        
        self.high_freq_branch = EnhancedHighFrequencyModel(power_dim, weather_dim, embed_dim, cnn_filters, cnn_kernel_size, n_gru_layers, num_heads, dropout, transformer_layers, num_pos_buckets)
        self.low_freq_branch = LowFrequencyLSTM(low_freq_dim, embed_dim, n_lstm_layers, dropout)
        
        self.fusion_mlp = nn.Sequential(nn.Linear(embed_dim * 2, embed_dim * 2), nn.GELU(), nn.Dropout(dropout))
        
        self.quantile_output_heads = nn.ModuleDict()
        for name in self.target_steps_names:
            self.quantile_output_heads[name] = nn.Linear(embed_dim * 2, len(self.quantiles_to_predict))

    def forward(self, x_high, x_weather, x_low):
        shared_high_freq_features = self.high_freq_branch(x_high, x_weather)
        shared_low_freq_features = self.low_freq_branch(x_low)
        combined_features = torch.cat([shared_high_freq_features, shared_low_freq_features], dim=1)
        final_features = self.fusion_mlp(combined_features)
        return_dict = {name: self.quantile_output_heads[name](final_features) for name in self.target_steps_names}
        return return_dict

# --- 分位数损失函数 ---
def quantile_loss(y_true, y_pred_quantiles, quantiles_to_predict):
    losses = []
    for i, q in enumerate(quantiles_to_predict):
        y_pred_q = y_pred_quantiles[:, i]
        errors = y_true - y_pred_q
        loss = torch.max(q * errors, (q - 1) * errors)
        losses.append(loss.mean())
    return torch.stack(losses).sum()

# =============================================================================
# 5. PSO 目标函数
# =============================================================================
def objective_function(params, train_loader, val_loader, device):
    """
    PSO的目标函数。接收一组超参数，返回模型在该超参数下的最小验证损失。
    """
    set_seed(GLOBAL_SEED) # 保证每次评估的独立性和可复现性
    
    try:
        # 动态调整 embed_dim (即 units) 以确保可以被 num_heads 整除
        embed_dim = params['units']
        if embed_dim % NUM_HEADS != 0:
            embed_dim = int(round(embed_dim / NUM_HEADS) * NUM_HEADS)
            if embed_dim == 0: embed_dim = NUM_HEADS
        
        model = ICIAL_UV_RPE_Model(
            target_steps_names=[f't_plus_{s}' for s in TARGET_STEPS],
            power_dim=HIGH_FREQ_FEATURES, weather_dim=WEATHER_FEATURES, low_freq_dim=LOW_FREQ_FEATURES,
            # --- 使用PSO传递的可调参数 ---
            embed_dim=embed_dim, 
            n_gru_layers=params['BiGRU_layers'],
            n_lstm_layers=params['LSTM_layers'],
            cnn_filters=params['CNN_filters'],
            cnn_kernel_size=params['cnn_kernel_size'],
            # --- 使用固定的超参数 ---
            num_heads=NUM_HEADS, 
            dropout=DROPOUT, 
            transformer_layers=TRANSFORMER_LAYERS,
            num_pos_buckets=RELATIVE_POSITION_BUCKETS
        ).to(device)
    except Exception as e:
        print(f"模型实例化失败! 参数: {params}, 错误: {e}")
        return float('inf') # 返回一个很大的值表示此参数组合无效

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # --- 缩短版的训练/验证循环 ---
    EVAL_EPOCHS = 10   # 每个粒子评估时只跑10轮
    EVAL_PATIENCE = 3  # 早停也更敏感
    
    best_val_loss = float('inf')
    early_stopping_counter = 0
    model_quantiles = model.quantiles_to_predict

    for epoch in range(EVAL_EPOCHS):
        model.train()
        for x_high, x_weather, x_low, *y_targets in train_loader:
            x_high, x_weather, x_low = x_high.to(device), x_weather.to(device), x_low.to(device)
            y_targets_on_device = [y.to(device) for y in y_targets]
            optimizer.zero_grad()
            predictions_dict = model(x_high, x_weather, x_low)
            batch_loss = 0.0
            for i, name in enumerate(model.target_steps_names):
                batch_loss += quantile_loss(y_targets_on_device[i], predictions_dict[name], model_quantiles)
            batch_loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x_high, x_weather, x_low, *y_targets in val_loader:
                x_high, x_weather, x_low = x_high.to(device), x_weather.to(device), x_low.to(device)
                y_targets_on_device = [y.to(device) for y in y_targets]
                predictions_dict = model(x_high, x_weather, x_low)
                batch_val_loss = 0.0
                for i, name in enumerate(model.target_steps_names):
                    batch_val_loss += quantile_loss(y_targets_on_device[i], predictions_dict[name], model_quantiles)
                val_loss += batch_val_loss.item()
        
        avg_val_loss = val_loss / len(val_loader)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= EVAL_PATIENCE:
                # print(f"提前停止于 epoch {epoch+1}，最佳验证损失: {best_val_loss:.4f}")
                break
    
    return best_val_loss

# =============================================================================
# 6. 主执行脚本
# =============================================================================
if __name__ == '__main__':
    # --- 步骤 1: 数据加载与准备 (只执行一次) ---
    print("--- 步骤1: 正在加载和预处理数据 ---")
    try:
        high_freq_df = pd.read_csv(HIGH_FREQ_DATA_PATH).interpolate()
        low_freq_df = pd.read_csv(LOW_FREQ_DATA_PATH).interpolate()
        original_df = pd.read_csv(ORIGINAL_DATA_PATH).interpolate()
    except FileNotFoundError as e:
        print(f"错误：找不到数据文件！请在脚本中更新文件路径。{e}")
        exit()

    weather_df = original_df[['Temp','Humidity','GHI','DHI','Rainfall']]
    power_df = original_df[['Power']]

    print("正在将时间序列转换为监督学习格式...")
    processed_power = time_series_to_supervised_mimo(power_df, N_STEPS_IN, N_STEPS_OUT)
    processed_high = time_series_to_supervised_mimo(high_freq_df, N_STEPS_IN, N_STEPS_OUT)
    processed_low = time_series_to_supervised_mimo(low_freq_df, N_STEPS_IN, N_STEPS_OUT)
    processed_weather = time_series_to_supervised_mimo(weather_df, N_STEPS_IN, N_STEPS_OUT)

    common_index = processed_power.index.intersection(processed_high.index).intersection(processed_low.index).intersection(processed_weather.index)
    processed_power, processed_high, processed_low, processed_weather = [df.loc[common_index] for df in [processed_power, processed_high, processed_low, processed_weather]]
    
    y_cols = [f'Power(t{"" if s-1==0 else f"+{s-1}"})' for s in TARGET_STEPS] 
    y = processed_power[y_cols].values

    X_high = processed_high[[c for c in processed_high.columns if '(t-' in c]].values.reshape(-1, N_STEPS_IN, HIGH_FREQ_FEATURES)
    X_weather = processed_weather[[c for c in processed_weather.columns if '(t-' in c]].values.reshape(-1, N_STEPS_IN, WEATHER_FEATURES)
    X_low = processed_low[[c for c in processed_low.columns if '(t-' in c]].values.reshape(-1, N_STEPS_IN, LOW_FREQ_FEATURES)

    train_size = int(len(y) * 0.8); val_size = int(len(y) * 0.15)
    def split_data(data): return data[:train_size], data[train_size:train_size+val_size], data[train_size+val_size:]
    train_X_high, val_X_high, test_X_high = split_data(X_high)
    train_X_weather, val_X_weather, test_X_weather = split_data(X_weather)
    train_X_low, val_X_low, test_X_low = split_data(X_low)
    train_y, val_y, test_y = split_data(y)

    scaler_high = MinMaxScaler(); scaler_weather = MinMaxScaler(); scaler_low = MinMaxScaler()
    def scale_3d_data(train, val, test, scaler):
        train_s = scaler.fit_transform(train.reshape(-1, train.shape[-1])).reshape(train.shape)
        val_s = scaler.transform(val.reshape(-1, val.shape[-1])).reshape(val.shape)
        test_s = scaler.transform(test.reshape(-1, test.shape[-1])).reshape(test.shape)
        return train_s, val_s, test_s
    train_X_high_s, val_X_high_s, test_X_high_s = scale_3d_data(train_X_high, val_X_high, test_X_high, scaler_high)
    train_X_weather_s, val_X_weather_s, test_X_weather_s = scale_3d_data(train_X_weather, val_X_weather, test_X_weather, scaler_weather)
    train_X_low_s, val_X_low_s, test_X_low_s = scale_3d_data(train_X_low, val_X_low, test_X_low, scaler_low)

    train_y_s_list, val_y_s_list = [], []
    for i, s in enumerate(TARGET_STEPS):
        scaler = MinMaxScaler()
        train_y_s_list.append(scaler.fit_transform(train_y[:, i:i+1]).flatten())
        val_y_s_list.append(scaler.transform(val_y[:, i:i+1]).flatten())

    train_y_tensors = [torch.from_numpy(v).float() for v in train_y_s_list]
    val_y_tensors = [torch.from_numpy(v).float() for v in val_y_s_list]

    train_data = TensorDataset(torch.from_numpy(train_X_high_s).float(), torch.from_numpy(train_X_weather_s).float(), torch.from_numpy(train_X_low_s).float(), *train_y_tensors)
    val_data = TensorDataset(torch.from_numpy(val_X_high_s).float(), torch.from_numpy(val_X_weather_s).float(), torch.from_numpy(val_X_low_s).float(), *val_y_tensors)
    
    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)
    print("数据准备完成。")

    # --- 步骤 2: 定义PSO搜索空间和参数 ---
    print("\n--- 步骤2: 配置PSO优化器 ---")
    # 格式: 'param_name': (min_value, max_value, type)
    param_space = {
        'CNN_filters': (30, 260, 'int'),
        'cnn_kernel_size': (1, 4, 'int'), # CNN_kernel_size in paper, use cnn_ for var name
        'LSTM_layers': (1, 4, 'int'),
        'units': (30, 260, 'int'), # Unified units for LSTM and BiGRU
        'BiGRU_layers': (1, 4, 'int'),
    }

    # PSO超参数
    N_PARTICLES = 100  # 粒子数量
    MAX_ITER = 100     # 迭代次数

    # --- 步骤 3: 实例化并运行PSO优化器 ---
    print("\n--- 步骤3: 开始运行PSO ---")
    pso = PSOOptimizer(
        objective_function=objective_function,
        param_space=param_space,
        n_particles=N_PARTICLES,
        max_iter=MAX_ITER,
        # 传递额外固定参数给目标函数
        train_loader=train_loader,
        val_loader=val_loader,
        device=device
    )
    
    best_params, best_score = pso.optimize()
    
    # --- 步骤 4: 显示最终结果 ---
    print("\n\n=============================================")
    print("           PSO 优化最终结果")
    print("=============================================")
    print(f"找到的最佳验证损失: {best_score}")
    print("找到的最佳超参数组合 (对应论文表格中的 Optimal Value):")
    print(f"  - CNN filters: {best_params.get('CNN_filters')}")
    print(f"  - CNN kernel_size: {best_params.get('cnn_kernel_size')}")
    print(f"  - LSTM layers: {best_params.get('LSTM_layers')}")
    print(f"  - LSTM units: {best_params.get('units')}")
    print(f"  - BiGRU layers: {best_params.get('BiGRU_layers')}")
    print(f"  - BiGRU units: {best_params.get('units')}") # BiGRU和LSTM共享units


随机数种子已设置为: 42
当前设备: cuda
--- 步骤1: 正在加载和预处理数据 ---
正在将时间序列转换为监督学习格式...
数据准备完成。

--- 步骤2: 配置PSO优化器 ---

--- 步骤3: 开始运行PSO ---
--- 开始PSO优化 ---

==================== PSO迭代: 1/50 ====================

--- [迭代 1/50 | 粒子 1/100] 正在评估参数: {'CNN_filters': 177, 'cnn_kernel_size': 1, 'LSTM_layers': 2, 'units': 81, 'BiGRU_layers': 3} ---
随机数种子已设置为: 42
